# <center>FIT5196- Assignment 1: Parsing Data and Text Preprocessing.</center>
## <center>Task 2: Text Preprocessing</center>
### Student ID: 29592461
### Student Name: Dinesh Karthikeyan
#### Language: Python 3.7.0
#### Working Environment: Jupyter Notebook
#### Packages Used: re, pandas,  numpy, nltk and itertools
#### Regex Checker: Pythex.
#### Overview of  methodology: 
* The data set to be parsed is Unit guide (in pdf format) for several units with synopsis and outcome, which is to be preprocessed and converted to count vector file and vocab file.
* The *29592461.pdf* document is converted to excel *29592461-converted.xlsx* using `https://smallpdf.com/pdf-to-excel` and the output excel is used to preprocess the text.
* First step is to read the contents from the file "*29592461-converted.xlsx*".
* The file content is loaded into data frame, the synopsis and outcomes are merged together and put in a different column.
* Tokenising the words from the column which has merged synopsis and outcomes and storing it in a dictionary based on the unit Id.
* The Tokens are used to get the bigrams.
* Top 200 bigrams are used based on the pmi measure and frequency filter 5.
* After finding the bigrams multi word tokenisation is done and stred in the dictionary.
* From this stop words from the file *stopwords_en.txt* is removed.
* After removing the stop words the more frequent (95% or 190 units), less frequent (5% or 10 units) and words with length less than three are removed.(based on the document frequency (occurence in diffrenet units.)).
* The dictionary with most frequent and less frequent removed is stemmed using porter stemmer.
* The words in this dictionary is the final vocabulary and it is written in the file *29592461_vocab.txt* with the index provided for each word (starting from 0).
* Now a Count vector file based on the final dictionary of words for units and the vocab file is created and written in the file *29592461_countVec.txt*.

#### Import Section

* re- For Regular Expression functionalities.
* numpy- For identifying Nan and removing.
* nltk- For all Natural Language tool kit functionalities.
* itertools- To convert values in dictionary to list.

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize import MWETokenizer
from nltk.collocations import *
from nltk.probability import *
import nltk
import itertools
from itertools import chain
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
import re

#### Text Preprocessing

In [2]:
read_file = pd.read_excel('29592461-converted.xlsx') # Loading the converted excel file into dataframe.

#### Case Normalisation 

* Converting the starting letters of the word of the sentences in the synopsis and outcome column.
* So as to maintain the Capital Tokens from the mid of the sentences.

In [3]:
for each in range(len(read_file)):    
    temp_Synopsis=sent_tokenize(read_file.Synopsis[each]) # Sentence tokenising the synopsis column values.
    for each_line in range(len(temp_Synopsis)):
        # Making the first letter to lower case.
        temp_Synopsis[each_line]=temp_Synopsis[each_line][0].lower()+temp_Synopsis[each_line][1:] 
        temp_Synopsis[each_line]=temp_Synopsis[each_line].replace("\n"," ") # Replacing unnecessary new line charaters.
        
    
    read_file.Synopsis[each]=" ".join(temp_Synopsis) # Grouping the synopsis again.

In [4]:
for each in range(len(read_file)):
    temp = read_file.Outcomes[each].split(",") # Converting the string which is in list format to list using split by ",".
    read_file.Outcomes[each]="".join(temp) # Removing split and converting list to paragraphs.
    # Removing unnecessary symbols.
    read_file.Outcomes[each] = read_file.Outcomes[each].strip("['") 
    read_file.Outcomes[each]= read_file.Outcomes[each].strip("']")
    read_file.Outcomes[each] = read_file.Outcomes[each].replace("'",'')
    read_file.Outcomes[each] = read_file.Outcomes[each].replace('"','')
    read_file.Outcomes[each] = read_file.Outcomes[each].replace(";",'. ')
    
for each in range(len(read_file)):
    if read_file.Outcomes[each]!='NA' :
        temp_outcome=sent_tokenize(read_file.Outcomes[each]) # Sentence tokenising the Outcomes column values.,
        for each_1 in range(len(temp_outcome)):
            temp_outcome[each_1]=re.sub(r'\'|\\n',' ',temp_outcome[each_1]).strip() # Replacing unnecessary new line characters. 
            # Making the first letter to lower case.
            temp_outcome[each_1]=temp_outcome[each_1][0].lower()+temp_outcome[each_1][1:]
            
    
        read_file.Outcomes[each]=" ".join(temp_outcome) # Grouping the Outcomes again.
read_file["Sy_Out"]=read_file.Synopsis+" "+read_file.Outcomes # Combining thew synopsis and outcomes into a new column.

#### Collecting data based on unit
* After combining the synopsis and outcomes into a new column a dictionary of thgese value with the unit id as key is created.
* If any duplicate unit id is present then its value is appended to the unit id which is already present.

In [5]:
unit_chunk_dict=dict()
for each in range(len(read_file.Title)):
    if read_file.Title[each] not in unit_chunk_dict.keys():
        unit_chunk_dict[read_file.Title[each]]=read_file.Sy_Out[each]
    else:
        unit_chunk_dict[read_file.Title[each]]+=read_file.Sy_Out[each]

#### Word Tokenisation

* Creating tokens based on the regex `\w+(?:[-']\w+)?` using RegexpTokenizer.
* These are the first unigram tokens formed for each unit and are stored in the dictionary based on the unit id.

In [6]:
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?")
unigram_tokens= dict()
for each in unit_chunk_dict.keys():
    unigram_tokens[each]=tokenizer.tokenize(unit_chunk_dict[each]) # Tokenising and storing into the dictionary.

#### Bigram Formation

* Using the unigram tokens formed above, with the help of nltk.collocations bigrams are formed with frequency of minimum 5 units, with the bigram measure of pmi and length of bigram greater than or equal to 3.
* Top 200 bigrams are generated and sent MWETokenizer for multi word tokenisation is done and stored into a dictionary based on the unit id.

In [7]:
words=[]
words=list(chain.from_iterable([value for value in unigram_tokens.values()]))
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(words)
bigram_finder.apply_freq_filter(5)
bigram_finder.apply_word_filter(lambda w: len(w) < 3)
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top-200 bigrams with pmi measure
mwetokenizer = MWETokenizer(top_200_bigrams) 
# Mwe Tokenisation and storing into dictionary.
colloc_values =  dict((uid, mwetokenizer.tokenize(value)) for uid,value in unigram_tokens.items()) 

#### Stop word removal

* Removing stop words in the stopwords file *stopwords_en.txt* from the dictionary of unigram and bigram tokens. 

In [8]:
stop_word_file=open("stopwords_en.txt")
stop_words=stop_word_file.read().splitlines()
stop_words_set=set([each for each in stop_words]) # Stop word set
vocab=[]
words_1=[]
words_1=list(chain.from_iterable(colloc_values.values()))
vocab = set(words_1)
for each_list in colloc_values.keys():
    colloc_values[each_list]=[word for word in colloc_values[each_list] if word not in stop_words_set] # Removing the stop words.

#### Removing Most Frequent, Less Frequent and Length less than 3 words

* With the values the dictionary after removing the stopwords the document frequency is calculated using FreqDist() function.
* If the frequency is less than 10 and greater than 190 (5% and 90% for 200 units) the words are removed.
* If the length of the word is less than 3 it is removed.


In [9]:
words_2=list(chain.from_iterable([set(value) for value in colloc_values.values()]))
fd_2 = FreqDist(words_2)
vocab=set(words_2)
for each in colloc_values.keys():
     colloc_values[each]=[word for word in colloc_values[each] if word in fd_2.keys() and fd_2[word]>= 10 and fd_2[word] <= 190 and len(word)>=3]

#### Stemming

* Porter Stemmer is used to stem the words.
* The word in the dictionary after the above step is stemmed to bring it to its root word.

In [10]:
ps=PorterStemmer()
for each in colloc_values.keys():    
    temp1=[]
    for each_word in colloc_values[each]:
        temp1.append(ps.stem(each_word))
    colloc_values[each]=temp1   

#### Forming Vocabulary

* After stemming, unique words from each unit is added into the list of vocabulary.

In [11]:
words_3=list(chain.from_iterable([set(value) for value in colloc_values.values()]))
vocab=set(words_3)
final_vocab=dict()
vocab=sorted(list(vocab))
for index in range(len(vocab)):
    final_vocab[index]=vocab[index]

#### Writing into vocabulary file

* The final vocabulary list from the above step is written into a vocab file *29592461_vocab.txt*
* The file holds the words in the vocabulary in sorted order along with the token index starting from 0.

In [12]:
vocab_file = open("29592461_vocab.txt", 'w')
for key in final_vocab.keys():
    vocab_file.write(final_vocab[key]+":"+str(key)+"\n")
vocab_file.close()

#### Writing Count Vector File

* Forming a final string to wite into counter vector file.
* the count vector file should have *UnitCode,Token_index1:Token1_count_in_this_unit,Token_index2:Token2_count_in_this_unit,.....*
* This final string is written into *29592461_countVec.txt*.

In [13]:
final_string=""
for unit in colloc_values.keys():
    final_string+=unit+","
    dict_keys=list(final_vocab.keys())
    for token_index in range(len(dict_keys)):
        if token_index!=len(dict_keys)-1:
            final_string+=str(dict_keys[token_index])+":"+str(colloc_values[unit].count(final_vocab[dict_keys[token_index]]))+","
        else:
            final_string+=str(dict_keys[token_index])+":"+str(colloc_values[unit].count(final_vocab[dict_keys[token_index]]))+"\n"
count_vector_file=open("29592461_countVec.txt","w")
count_vector_file.write(final_string)
count_vector_file.close()     